### 1) Import the libraries

In [4]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
import re
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
import sys
from gensim.models import Word2Vec
from scipy import spatial
import networkx as nx

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ishan_borker\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ishan_borker\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


AttributeError: module 'lib' has no attribute 'X509_V_FLAG_CB_ISSUER_CHECK'

### 2) Load the text to summarize

In [ ]:
text='''Santiago is a Shepherd who has a recurring dream which is supposedly prophetic. Inspired on learning this, he undertakes a journey to Egypt to discover the meaning of life and fulfill his destiny. During the course of his travels, he learns of his true purpose and meets many characters, including an “Alchemist”, that teach him valuable lessons about achieving his dreams. Santiago sets his sights on obtaining a certain kind of “treasure” for which he travels to Egypt. The key message is, “when you want something, all the universe conspires in helping you to achieve it.” Towards the final arc, Santiago gets robbed by bandits who end up revealing that the “treasure” he was looking for is buried in the place where his journey began. The end.'''
print(text)

### 3) Tokenizing this paragraph into a list of sentences

In [ ]:
sentences=sent_tokenize(text)
print(sentences)

### 4) Preprocessing 

In [ ]:
sentences_clean=[re.sub(r'[^\w\s]','',sentence.lower()) for sentence in sentences]
print(sentences_clean)
stop_words = stopwords.words('english')
print(stop_words)
sentence_tokens=[[words for words in sentence.split(' ') if words not in stop_words] for sentence in sentences_clean]
print(sentence_tokens)

### 5) Calculate word embedding and sentence embedding using sentence_tokens

In [ ]:
w2v=Word2Vec(sentence_tokens,size=1,min_count=1,iter=1000)
print(w2v)
sentence_embeddings=[[w2v[word][0] for word in words] for words in sentence_tokens]
print(sentence_embeddings)
max_len=max([len(tokens) for tokens in sentence_tokens])
print(max_len)
sentence_embeddings=[np.pad(embedding,(0,max_len-len(embedding)),'constant') for embedding in sentence_embeddings]
print(sentence_embeddings)

### 6) Calculate the similarity matrix

In [ ]:
similarity_matrix = np.zeros([len(sentence_tokens), len(sentence_tokens)])
for i,row_embedding in enumerate(sentence_embeddings):
    for j,column_embedding in enumerate(sentence_embeddings):
        similarity_matrix[i][j]=1-spatial.distance.cosine(row_embedding,column_embedding)
print(similarity_matrix)

### 7) Implement PageRank

In [ ]:
nx_graph = nx.from_numpy_array(similarity_matrix)
scores = nx.pagerank(nx_graph)
print(scores)

### 8) Selecting Top 4 sentences for summary

In [ ]:
top_sentence={sentence:scores[index] for index,sentence in enumerate(sentences)}
print(top_sentence)
top=dict(sorted(top_sentence.items(), key=lambda x: x[1], reverse=True)[:4])
print(top)

In [ ]:
for sent in sentences:
    if sent in top.keys():
        print(sent)